In [1]:
#This is the skip gram model which is a naive model for understanding NLP basics
import math 
import numpy as np
import tensorflow as tf

'''
Considering the following statement :
"the first coding homework was fun"

with a window size of 1 ,we have the dataset:
([the,coding],first),([lot,fun],of)....

The skipgram model tries to predict that each context word from its target word,
and so the task becomes to predict 'the 'and 'coding' from first.'lot'
and 'fun' from 'of' and so on.

The dataset now is:
(first,the),(first,coding),(of,lot),(of,fun)....
'''


'\nConsidering the following statement :\n"the first coding homework was fun"\n\nwith a window size of 1 ,we have the dataset:\n([the,coding],first),([lot,fun],of)....\n\nThe skipgram model tries to predict each context word from its target worf,\nand so the task becomes to predict \'the \'and \'coding\' from first.\'lot\'\nand \'fun\' from \'of\' and so on.\n\nThe dataset now is:\n(first,the),(first,coding),(of,lot),(of,fun)....\n'

In [3]:
#defining the constants
batch_size=128
vocabulary_size=50000
embedding_size=128 #dimensuon of the embedded vector
num_sampled=64 #Numbe rof negative examples to sample
'''
load_data loads the already preprocessed training and val data

train_data is a list of {batch_input,batch_labels} pairs.
val data is a list of all validation inputs.
reverse_dictionary is a python dict from word index to word
'''

train_data,val_data,reverse_dictionary=load_data()
print("Number of training examples:",len(train_data)*batch_size)
print("Number of validation examples:",len(val_data))

def skipgram():
    batch_inputs=tf.placeholder(tf.int32,shape=[batch_size,])
    batch_labels=tf.placeholder(tf.int32,shape=[batch_size,1])
    val_dataset=tf.constant(val_data,dtype=tf.int32)
    
    
    with tf.variable_scope("word2vec") as scope:
        embeddings=tf.Variable(tf.random_uniform([vocabulary_size,
                                                 embedding_size],
                                                -1.0,1.0))
        batch_embeddings=tf.nn.embedding_lookup(embeddings,batch_inputs)
        
        
        weights=tf.Variable(tf.truncated_normal([vocabulary_size,
                                                embedding_size],
                            stddev=1.0/math.sqrt(embedding_size)))
        biases=tf.Variable(tf.zeros([vocabulary_size]))
        
        loss=tf.reduce_mean(tf.nn.nce_loss(weights=weights,
                           biases=biases,
                           labels=batch_labels,
                           inputs=batch_inputs,
                           num_sampled=num_sampled,
                           num_classes=vocabulary_size))
        '''
        num_sampled for automatic addition of context and noisy samples instead of only
        context words and target words.num_sampled=64, will look up 64 words which are not 
        in our sample training set and which are noise words and serve as negative samples 
        so that network knows which words are actaully 
        context words and which are not
        '''
         
        
        norm=tf.sqrt(tf.reduce_mean(tf.square(embeddings),1,keep_dims=True))
        normalized_embeddings=embedding/norm
        
        val_embeddings=tf.nn.embedding_lookup(normalized_embeddings,val_dataset)
        similarity=tf.matmul(val_embeddings,normalized_embeddings,tranpose_b=True)
        
        return batch_inputs,batch_labels,normalized_embeddings,loss,similarity
    
            
    
    
    
    
    
def run():
    
    batch_inputs,batch_labels,normalized_embeddings,loss,similarity=skipgram()
    optimizer=tf.train.GradientDescentOptimizer(1.0).minimize(loss)
    
    
    
    init=tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        
        average_loss=0.0
        for step ,batch_data in enumerate(train_data):
            inputs,labels=batch_data
            feed_dict={batch_inputs:inputs,batch_labels:labels}
            
            _,loss_val=sess.run([optimizer,loss],feed_dict)
            avearge_loss+=loss_val
            
            if step%1000==0:
                if step>0:
                    average_loss/=1000
                    print('loss at iteration',step,":",average_loss)
                    average_loss=0
                    
            if step%5000==0:
                sim=similarity.eval()  #similar to call session.run on similarity and fetching back
                for i in xrange(len(val_data)):
                    top_k=8
                    nearest=(sim[i,:]).argsort()[1:top_k+1]
                    print_closest_words(val_data[i],nearest,reverse_dictionary)
         
        final_embeddings= normalized_embedding.eval()          
                    
                
                
        
    
#lets start training
final_embeddings=run()
    
#visualizinng the embeddings
visualize_embeddings(final_embeddings,reverse_dictionary)
